# Source Detection Summary

Explore how many sources are detected as a function of, e.g., image quality, real/bogus score, etc.

## Connect to database

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
# from IPython.display import display
# import ipywidgets

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

from copy import deepcopy
import time

In [ ]:
font = {'size' : 15}
mpl.rc('font', **font)

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
db.autocommit = True # Makes the computer nicer about SQL syntax errors

In [ ]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
# tables = ['exposures','subtractions','objects','candidates']
# for table in tables:
#     query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
#     cursor.execute( query, ( table, ))
#     print( f"\nTABLE: {table}\n===========================" )
#     for row in cursor:
#         print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

# tables = ['subtractions']
# for table in tables:
#     query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
#     cursor.execute( query, ( table, ))
#     print( f"\nTABLE: {table}\n===========================" )
#     for row in cursor:
#         print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

## NOIRLab Archive Image List

Melissa made this list of all 2021A-0113 and 2021B-0149 images from the NOIRLab Data Archive.

In [ ]:
fnm = 'archive_image_list.txt'
arch_fbase   = np.loadtxt( fnm, dtype='str', usecols=(0) )
arch_ra      = np.loadtxt( fnm, dtype='float', usecols=(1) )
arch_dec     = np.loadtxt( fnm, dtype='float', usecols=(2) )
arch_mjd     = np.loadtxt( fnm, dtype='float', usecols=(3) )
arch_filt    = np.loadtxt( fnm, dtype='str', usecols=(4) )
arch_airm    = np.loadtxt( fnm, dtype='float', usecols=(5) )
arch_obj     = np.loadtxt( fnm, dtype='str', usecols=(6) )
arch_expt    = np.loadtxt( fnm, dtype='float', usecols=(7) )
arch_seqid   = np.loadtxt( fnm, dtype='str', usecols=(8) )
arch_class   = np.loadtxt( fnm, dtype='str', usecols=(9) )
arch_ftype   = np.loadtxt( fnm, dtype='str', usecols=(10) )
arch_moonsep = np.loadtxt( fnm, dtype='float', usecols=(11) )
arch_moonill = np.loadtxt( fnm, dtype='float', usecols=(12) )
arch_mpval   = np.loadtxt( fnm, dtype='float', usecols=(13) )

#### get rid of the DECaPS and non-standard archive images

In [ ]:
tx = np.where( (arch_class != 'COSMOS') & (arch_class != 'ELAIS') )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(arch_class))

arch_fbase   = np.delete( arch_fbase, tx )
arch_ra      = np.delete( arch_ra, tx )
arch_dec     = np.delete( arch_dec, tx )
arch_mjd     = np.delete( arch_mjd, tx )
arch_filt    = np.delete( arch_filt, tx )
arch_airm    = np.delete( arch_airm, tx )
arch_obj     = np.delete( arch_obj, tx )
arch_expt    = np.delete( arch_expt, tx )
arch_seqid   = np.delete( arch_seqid, tx )
arch_class   = np.delete( arch_class, tx )
arch_ftype   = np.delete( arch_ftype, tx )
arch_moonsep = np.delete( arch_moonsep, tx )
arch_moonill = np.delete( arch_moonill, tx )
arch_mpval   = np.delete( arch_mpval, tx )
del tx

print('new length of arch arrays = ',len(arch_fbase))

## Exposures and Subtractions Tables

### exposures

In [ ]:
query = "SELECT id, ra, dec, filename, mjd, filter, proposalid, header FROM exposures"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
exp_id  = np.asarray( results[0], dtype='int' )
exp_ra  = np.asarray( results[1], dtype='float' )
exp_dec = np.asarray( results[2], dtype='float' )
exp_fnm = np.asarray( results[3], dtype='str' )
exp_mjd = np.asarray( results[4], dtype='float' )
exp_fil = np.asarray( results[5], dtype='str' )
exp_pid = np.asarray( results[6], dtype='str' )
exp_hdr = np.asarray( results[7], dtype='str' )
del query, results

In [ ]:
### exp_fbs : just the filename base (without the extension)
### exp_cal : calendar date
exp_fbs = deepcopy(exp_fnm)
exp_cal = np.zeros( len(exp_fnm), dtype='int' )

for i,fnm in enumerate(exp_fnm):
    tmp1 = fnm.split('.')[0]
    exp_fbs[i] = tmp1
    tmp2 = tmp1.split('_')[1]
    exp_cal[i] = int(tmp2)
    del tmp1,tmp2

# print('fnm: ', exp_fnm[0])
# print('fbs: ',exp_fbs[0])
# print('cal: ',exp_cal[0])

#### get rid of non-DDF and non-standard exposures

In [ ]:
### exp_aid : archive identifier (index in archive array)
exp_aid = np.zeros( len(exp_id), dtype='int' ) - 1

tag = 0
for i,fbase in enumerate(arch_fbase):
    tx = np.where( fbase == exp_fbs )[0]
    if len(tx) == 1:
        exp_aid[tx[0]] = i
    elif len(tx) > 1:
        exp_aid[tx[0]] = i
        tag += 1
    del tx

print(tag,' cases of double-matches, only the first will be kept')
del tag

In [ ]:
tx = np.where( exp_aid == -1 )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(exp_aid))

exp_id = np.delete( exp_id, tx )
exp_ra = np.delete( exp_ra, tx )
exp_dec = np.delete( exp_dec, tx )
exp_fnm = np.delete( exp_fnm, tx )
exp_mjd = np.delete( exp_mjd, tx )
exp_fil = np.delete( exp_fil, tx )
exp_pid = np.delete( exp_pid, tx )
exp_hdr = np.delete( exp_hdr, tx )
exp_fbs = np.delete( exp_fbs, tx )
exp_cal = np.delete( exp_cal, tx )
exp_aid = np.delete( exp_aid, tx )
del tx

print('new length of exp arrays = ',len(exp_id))

### subtractions

In [ ]:
query = "SELECT id, ra, dec, lmt_mg, seeing, skysig, magzp, "+ \
        "ccdnum, image_id, exposure_id FROM subtractions"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
sub_id  = np.asarray( results[0], dtype='int' )
sub_ra  = np.asarray( results[1], dtype='float' )
sub_dec = np.asarray( results[2], dtype='float' )
sub_lmg = np.asarray( results[3], dtype='float' )
sub_see = np.asarray( results[4], dtype='float' )
sub_sks = np.asarray( results[5], dtype='float' )
sub_mzp = np.asarray( results[6], dtype='float' )
sub_ccd = np.asarray( results[7], dtype='int' )
sub_iid = np.asarray( results[8], dtype='int' )
sub_eid = np.asarray( results[9], dtype='int' )
del query, results

#### get rid of subtractions that aren't matched to exposure table

In [ ]:
### sub_expind : exposure table index
sub_expind = np.zeros( len(sub_id), dtype='int' ) - 1

tag = 0
for i in range(len(exp_id)):
    tx = np.where( exp_id[i] == sub_eid )[0]
    if len(tx) > 0:
        sub_expind[tx] = i
    else:
        tag += 1
    del tx

print(tag,' exposures had no subtractions')
del tag

In [ ]:
tx = np.where( sub_expind == -1 )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(sub_id))
sub_id  = np.delete(sub_id, tx)
sub_ra  = np.delete(sub_ra, tx)
sub_dec = np.delete(sub_dec, tx)
sub_lmg = np.delete(sub_lmg, tx)
sub_see = np.delete(sub_see, tx)
sub_sks = np.delete(sub_sks, tx)
sub_mzp = np.delete(sub_mzp, tx)
sub_ccd = np.delete(sub_ccd, tx)
sub_iid = np.delete(sub_iid, tx)
sub_eid = np.delete(sub_eid, tx)
sub_expind = np.delete(sub_expind, tx)

print('new length of sub arrays = ',len(sub_id))
del tx

## Objects and Candidates Tables

**objects** are the individual detections on a subtraction <br>
**candidates** are objects associated by location <br>

### objects

this query takes about 80 seconds

In [ ]:
t1 = time.time()

query = ("SELECT o.id, o.ra, o.dec, o.candidate_id, o.subtraction_id, rbs.rb, o.mag, o.magerr, o.ignore FROM objects o "
         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=1 ")
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

obj_id     = np.asarray( results[0], dtype='int' )
obj_ra     = np.asarray( results[1], dtype='float' )
obj_dec    = np.asarray( results[2], dtype='float' )
obj_candid = np.asarray( results[3] )
obj_subid  = np.asarray( results[4], dtype='int' )
obj_rb     = np.asarray( results[5], dtype='float' )
obj_mag    = np.asarray( results[6], dtype='float' )
obj_magerr = np.asarray( results[7], dtype='float' )
obj_ignore = np.asarray( results[8] )
del query, results

print(len(obj_id))

obj_ignore is always false

In [ ]:
# print(obj_ignore)
# tx = np.where( obj_ignore == 'True' )[0]
# print(len(tx))
# del tx

#### get rid of objects that are not matched to a subtraction

In [ ]:
### Check that all sub_id are unique (all lengths are the same)
values, indices, counts = np.unique( sub_id, return_inverse = True, return_counts = True )
print( 'sub_id lengths = ', len(values), len(indices), len(counts) )
print( values )
del values, indices, counts

### Note that obj_subid are not unique, of course
values, indices, counts = np.unique( obj_subid, return_inverse = True, return_counts = True )
print( 'obj_subid lengths = ', len(values), len(indices), len(counts) )
print( values )
del values, indices, counts

In [ ]:
fout = open('deleteme.txt','w')

obj_subind = np.zeros( len(obj_subid), dtype='int' ) - 1
sub_nobj = np.zeros( len(sub_id), dtype='int' )
sub_nobjg = np.zeros( len(sub_id), dtype='int' )

ssx = np.argsort( sub_id )
ossx = np.argsort( obj_subid )

t1 = time.time()

j = 0
for i in range(len(ssx)):
    cntr = 0
    cntrg = 0
    
    if (i == 1000) | (i == 10000):
        timeleft = ((time.time()-t1)/float(i)) * (len(sub_id)-float(i))
        print('TIMER: i=',i,'  this will take another ',timeleft,' seconds')
        del timeleft
    
    next_sub = False
    while (next_sub == False) & (j < len(ossx)):
        cntr += 1
        if obj_rb[ossx[j]] > 0.6:
            cntrg += 1
        if sub_id[ssx[i]] > obj_subid[ossx[j]]:
            fout.write('%1s %10i %10i %10s %10i %10i \n' %\
                       ('A', i, j, next_sub, sub_id[ssx[i]], obj_subid[ossx[j]]) )
            j += 1
        elif sub_id[ssx[i]] == obj_subid[ossx[j]]:
            obj_subind[ossx[j]] = ssx[i]
            fout.write('%1s %10i %10i %10s %10i %10i \n' %\
                       ('B', i, j, next_sub, sub_id[ssx[i]], obj_subid[ossx[j]]) )
            j += 1
        elif sub_id[ssx[i]] < obj_subid[ossx[j]]:
            next_sub = True
            fout.write('%1s %10i %10i %10s %10i %10i \n' %\
                       ('C', i, j, next_sub, sub_id[ssx[i]], obj_subid[ossx[j]]) )

    sub_nobj[ssx[i]] = cntr
    sub_nobjg[ssx[i]] = cntrg
    del cntr, cntrg

fout.close()
print('wrote to deleteme.txt')

del i, j, ssx, ossx

t2 = time.time()
print(t2-t1)
del t1,t2

In [ ]:
os.system('rm deleteme.txt')

now actually remove the elements of the objects array

In [ ]:
tx = np.where( obj_subind < 0 )[0]
print('identified ',len(tx),' array elements to remove, out of ',len(obj_subind))

obj_id     = np.delete( obj_id, tx )
obj_ra     = np.delete( obj_ra, tx )
obj_dec    = np.delete( obj_dec, tx )
obj_candid = np.delete( obj_candid, tx )
obj_subid  = np.delete( obj_subid, tx )
obj_rb     = np.delete( obj_rb, tx )
obj_mag    = np.delete( obj_mag, tx )
obj_magerr = np.delete( obj_magerr, tx )
obj_ignore = np.delete( obj_ignore, tx )
obj_subind = np.delete( obj_subind, tx )

print('new length of obj arrays = ',len(obj_id))
del tx

#### update the exposures table with subtraction data

In [ ]:
### mean limiting magnitudes, seeing, and sky background
exp_mlmg = np.zeros( len(exp_id), dtype='float')
exp_msee = np.zeros( len(exp_id), dtype='float')
exp_msks = np.zeros( len(exp_id), dtype='float')

### total number of objects, and good objects with R/B>0.6
exp_tnobj = np.zeros( len(exp_id), dtype='float')
exp_tnobjg = np.zeros( len(exp_id), dtype='float')

t1 = time.time()

for e,expid in enumerate(exp_id):
    if (e == 100) | (e == 1000) | (e == 10000):
        t2 = time.time()
        print('time remaining: ', ((t2-t1)/float(e))*(float(len(exp_id))-float(e)) )
    tx = np.where( sub_eid == expid )[0]
    exp_mlmg[e] = np.mean( sub_lmg[tx] )
    exp_msee[e] = np.mean( sub_see[tx] )
    exp_msks[e] = np.mean( sub_sks[tx] )
    exp_tnobj[e] = np.sum( sub_nobj[tx] )
    exp_tnobjg[e] = np.sum( sub_nobjg[tx] )
    del tx

t3 = time.time()
print('done, ',t3-t1,' elapsed')
del t1,t2,t3

#### get rid of objects that are duplicates

they are within 0.1" of another object in the same subtraction <br>
this would take about 125 minutes, need to find a better way <br>
used to be done in a small amount of time but now there are too many objects 

In [ ]:
# t1 = time.time()

# obj_dup = np.zeros( len(obj_id), dtype='int' )

# limit = (0.1 / 3600.0)**2
# for s,subid in enumerate(sub_id):
#     if (s==100) | (s==1000) | (s==10000) | (s==30000) | (s==60000):
#         ix = np.where( obj_dup == 1 )[0]
#         print( 's = ', s, np.round(time.time()-t1,3), 's elapsed, ', len(ix),' duplicates identified' )
#         del ix
#     # all the objects in this subtraction
#     ox = np.where( (obj_subid == subid) & (obj_dup == 0) )[0]
#     for o in ox:
#         # all the objects in this subtraction with similar RA and Dec to the o'th in ox
#         tx = np.where( (obj_ra[ox]-obj_ra[o])**2 + (obj_dec[ox]-obj_dec[o])**2 < limit )[0]
#         if len(tx) > 0:
#             # identify all the matching objects as duplicates
#             obj_dup[ox[tx]] = 1
#             # except the first one, of course, consider it the 'real' one
#             obj_dup[o] = 0
#         del tx
#     del ox

# tx = np.where( obj_dup == 1 )[0]
# print('total number of duplicates identified: ',len(tx))
# del tx

# t2 = time.time()
# print(t2-t1)
# del t1,t2

In [ ]:
# tx = np.where( obj_dup == 1 )[0]
# print('identified ',len(tx),' array elements to remove, out of ',len(obj_tmp))

# obj_id     = np.delete( obj_id, tx )
# obj_ra     = np.delete( obj_ra, tx )
# obj_dec    = np.delete( obj_dec, tx )
# obj_candid = np.delete( obj_candid, tx )
# obj_subid  = np.delete( obj_subid, tx )
# obj_rb     = np.delete( obj_rb, tx )
# obj_mag    = np.delete( obj_mag, tx )
# obj_magerr = np.delete( obj_magerr, tx )
# obj_ignore = np.delete( obj_ignore, tx )

# print('new length of obj arrays = ',len(obj_id))
# del obj_tmp, tx

### candidates

this query takes about 30 seconds

In [ ]:
t1 = time.time()

# query = "SELECT id, ra, dec, nmatches FROM candidates"
query = "SELECT id, ra, dec FROM candidates"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

cand_id       = np.asarray( results[0] )
cand_ra       = np.asarray( results[1], dtype='float' )
cand_dec      = np.asarray( results[2], dtype='float' )
# cand_nmatches = np.asarray( results[3] )
del query, results

print(len(cand_id))

In [ ]:
### Check that all cand_id are unique (all lengths are the same)
values, indices, counts = np.unique( cand_id, return_inverse = True, return_counts = True )
print( 'lengths = ', len(values), len(indices), len(counts) )
del values, indices, counts

#### get rid of candidates that are not matched to an object

this process takes about 4 seconds

In [ ]:
values, indices, counts = np.unique( obj_candid, return_inverse = True, return_counts = True )
print( 'lengths = ', len(values), len(indices), len(counts) )
print( 'values = ', values )
print( 'indices = ', indices )
print( 'counts = ', counts )
print( ' ' )

t1 = time.time()

cand_nobj = np.zeros( len(cand_id), dtype='int' ) - 1

sx = np.argsort( cand_id )
cntr = 0
for c,val in enumerate( values ):
    match = False
    while match == False:
        if val == cand_id[sx[cntr]]:
            cand_nobj[sx[cntr]] = counts[c]
            cntr += 1
            match = True
        else:
            cntr += 1
    del match
del cntr,sx

t2 = time.time()
print(t2-t1)
del t1,t2

del values, indices, counts

In [ ]:
### That was really fast; double check.

# tx = np.where( cand_nobj <= 0 )[0]
# print(len(tx),' candidates are NOT matched to an object, out of ',len(cand_nobj))
# for i in range(10):
#     rx = np.where( obj_candid == cand_id[tx[i]] )[0]
#     print( cand_id[tx[i]], len(rx) )
#     del rx
# del tx

# tx = np.where( cand_nobj > 0 )[0]
# print(len(tx),' candidates ARE matched to an object, out of ',len(cand_nobj))
# for i in range(10):
#     rx = np.where( obj_candid == cand_id[tx[i]] )[0]
#     print( cand_id[tx[i]], len(rx) )
#     del rx
# del tx

In [ ]:
tx = np.where( cand_nobj <= 0 )[0]
cand_id   = np.delete( cand_id, tx )
cand_ra   = np.delete( cand_ra, tx )
cand_dec  = np.delete( cand_dec, tx )
cand_nobj = np.delete( cand_nobj, tx )

print('new length of cand arrays = ',len(cand_id))
del tx

### calculate candidate mean real/bogus score 

In [ ]:
t1 = time.time()

csx = np.argsort(cand_id)
osx = np.argsort(obj_candid)

cand_meanrb = np.zeros( len(cand_id), dtype='float' )

j = 0
for i,x in enumerate(csx):
    if (i==100) | (i==10000) | (i==100000):
        elt = time.time()-t1
        print('Time remaining:', (elt/float(i)) * float(len(cand_id)-float(i)) )
    
    if cand_id[x] == obj_candid[osx[j]]:
        match = True
        temp = []

        while match == True:
            if j < len(obj_candid):
                if cand_id[x] == obj_candid[osx[j]]:
                    temp.append( obj_rb[osx[j]] )
                    j += 1
                elif cand_id[x] != obj_candid[osx[j]]:
                    match = False
                    cand_meanrb[x] = np.mean( np.asarray( temp, dtype='float' ) )
            else:
                match = False
                cand_meanrb[x] = np.mean( np.asarray( temp, dtype='float' ) )

        del temp, match
        
t2 = time.time()
print('Time elapsed: ',t2-t1)
del t1,t2

del csx, osx

<br>
<br>
<br>

# Plots

In [ ]:
### COSMOS g, r, i, ELAIS g, r, i
c = ['darkgreen', 'firebrick', 'saddlebrown', 'limegreen', 'orange', 'lightcoral']

### COSMOS, ELIAS
cc = 'dodgerblue'
ce = 'darkviolet'

# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "sans-serif",
#     "font.sans-serif": ["Helvetica"]})

<br>
<br>

## plot exposure qualities

In [ ]:
cex = np.where( (exp_ra > 140.0) & (exp_ra < 160.0) & (exp_dec > -5.0)  & (exp_dec < 10.0) )[0]
eex = np.where( (exp_ra > 0.0)   & (exp_ra < 20.0)  & (exp_dec > -50.0) & (exp_dec < -40.0) )[0]
print('Number of COSMOS exposures: ',len(cex))
print('Number of ELAIS exposures: ',len(eex))
print(len(cex)+len(eex),' should equal ',len(exp_ra))

tx = np.where( exp_tnobj < 1 )[0]
print('Number of exposures with no objects: ',len(tx))
del tx

tx = np.where( exp_tnobjg < 1 )[0]
print('Number of exposures with no good objects: ',len(tx))
del tx

In [ ]:
exp_moonsep = arch_moonsep[ exp_aid ]
exp_moonill = arch_moonill[ exp_aid ]

In [ ]:
fig, ax = plt.subplots( 2, 3, figsize=(15,5), sharey=True )

ax[0,0].set_ylabel('Num. of Exp.')
ax[1,0].set_ylabel('Num. of Exp.')

for i,ifilt in enumerate(['g','r','i']):
    cfx = np.where( exp_fil[cex] == ifilt )[0]
    efx = np.where( exp_fil[eex] == ifilt )[0]
    j = i + 3
    ax[0,i].hist( exp_tnobj[cex[cfx]], bins=40, range=(0,4000), histtype='step', lw=3, alpha=0.5, \
                 color=c[i], label='COSMOS, '+ifilt+'-band')
    ax[0,i].hist( exp_tnobj[eex[efx]], bins=40, range=(0,4000), histtype='step', lw=3, alpha=0.5, \
                 color=c[j], label='ELAIS, '+ifilt+'-band')
    ax[0,i].set_xlabel(r'Total $N_{\rm obj}$ (all)')
    # ax[0,i].set_xlim([20,25])
    ax[0,i].legend(loc='upper right', labelspacing=0, fontsize=12)

for i,ifilt in enumerate(['g','r','i']):
    cfx = np.where( exp_fil[cex] == ifilt )[0]
    efx = np.where( exp_fil[eex] == ifilt )[0]
    j = i + 3
    ax[1,i].hist( exp_tnobjg[cex[cfx]], bins=40, range=(0,500), histtype='step', lw=1, color=c[i])
    ax[1,i].hist( exp_tnobjg[eex[efx]], bins=40, range=(0,500), histtype='step', lw=1, color=c[j])
    ax[1,i].set_xlabel(r'Total $N_{\rm obj}$ (R/B>0.6)')
    # ax[1,i].set_xlim([20,25])

ax[0,i].yaxis.set_tick_params(which='both', labelbottom=True)
# ax[1,i].yaxis.set_tick_params(which='both', labelbottom=True)

fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots( 5, 3, figsize=(15,17), sharey=True )

for row in range(5):
    for i,ifilt in enumerate(['g','r','i']):
        cfx = np.where( exp_fil[cex] == ifilt )[0]
        efx = np.where( exp_fil[eex] == ifilt )[0]
        j = i + 3

        if row == 0:
            ax[row,i].plot( exp_mlmg[cex[cfx]], np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[i])
            ax[row,i].plot( exp_mlmg[eex[efx]], np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[j])
            ax[row,i].set_xlabel("Mean Limiting Magnitude")
            ax[row,i].set_xlim([20,25])
            
            ax[row,i].plot( -99, -99, 'o', ms=6, alpha=1, mew=0, color=c[i], label='COSMOS, '+ifilt+'-band')
            ax[row,i].plot( -99, -99, 'o', ms=6, alpha=1, mew=0, color=c[j], label='ELAIS, '+ifilt+'-band')
            ax[row,i].legend(loc='upper left', handletextpad=0, labelspacing=0.1, fontsize=14)
        if row == 1:
            ax[row,i].plot( exp_msee[cex[cfx]], np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[i])
            ax[row,i].plot( exp_msee[eex[efx]], np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[j])
            ax[row,i].set_xlabel("Mean Seeing")
            ax[row,i].set_xlim([0.5,2.5])
        if row == 2:
            ax[row,i].plot( np.log10(exp_msks[cex[cfx]]), np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[i])
            ax[row,i].plot( np.log10(exp_msks[eex[efx]]), np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[j])
            ax[row,i].set_xlabel("Mean Sky Background")
            ax[row,i].set_xlim([0,2.5])
        if row == 3:
            ax[row,i].plot( exp_moonsep[cex[cfx]], np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[i])
            ax[row,i].plot( exp_moonsep[eex[efx]], np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[j])
            ax[row,i].set_xlabel("Moon Separation [degrees]")
            # ax[row,i].set_xlim([0,5000])
        if row == 4:
            ax[row,i].plot( exp_moonill[cex[cfx]], np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[i])
            ax[row,i].plot( exp_moonill[eex[efx]], np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.4, mew=0, color=c[j])
            ax[row,i].set_xlabel("Moon Illumination Fraction")
            # ax[row,i].set_xlim([0,5000])
        # if row == 5:
        #     ax[row,i].plot( exp_mjd[cex[cfx]], np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.2, mew=0, color=c[i])
        #     ax[row,i].plot( exp_mjd[eex[efx]], np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.2, mew=0, color=c[j])
        #     ax[row,i].set_xlabel("MJD")
        # if row == 6:
        #     ax[row,i].plot( exp_tnobj[cex[cfx]], np.log10(exp_tnobjg[cex[cfx]]), 'o', ms=4, alpha=0.2, mew=0, color=c[i])
        #     ax[row,i].plot( exp_tnobj[eex[efx]], np.log10(exp_tnobjg[eex[efx]]), 'o', ms=4, alpha=0.2, mew=0, color=c[j])
        #     ax[row,i].set_xlabel("Number of Objects (all)")
        #     ax[row,i].set_xlim([0,5000])

        ax[row,i].yaxis.set_tick_params(which='both', labelbottom=True)
        if i == 0:
            ax[row,i].set_ylabel(r'$N_{\rm obj}$'+' (R/B>0.6)')
        ax[row,i].set_ylim([1,3])

        del cfx,efx,j
    
fig.tight_layout()
plt.savefig('source_detection_summary_figures/srcdet_nobjimg')
# plt.show()

<br>
<br>

## plot subtraction qualities

Not needed, we did with the exposures instead.

In [ ]:
# csx = np.where( (sub_ra > 140.0) & (sub_ra < 160.0) & (sub_dec > -5.0)  & (sub_dec < 10.0) )[0]
# esx = np.where( (sub_ra > 0.0)   & (sub_ra < 20.0)  & (sub_dec > -50.0) & (sub_dec < -40.0) )[0]
# print('Number of COSMOS subtractions: ',len(csx))
# print('Number of ELAIS subtractions: ',len(esx))

In [ ]:
# sub_moonsep = arch_moonsep[ exp_aid[ sub_expind ] ]
# sub_moonill = arch_moonill[ exp_aid[ sub_expind ] ]

#### Which subtractions have no good objects? (R/B>0.6)

Plot fraction of objects that are good vs. various subtraction parameters.

There are some subtractions with no good objects, but it's not like a certain date, or certain CCD.

There are no subtractions with NO objects at all.

And since the subtractions only have up to about 100 objects each, it's not that weird that some diffs would be empty.

In [ ]:
# fig, ax = plt.subplots( 6, 3, figsize=(15,20), sharey=True )

# for row in range(6):
#     for i,ifilt in enumerate(['g','r','i']):
#         cfx = np.where( exp_fil[ sub_expind[csx] ] == ifilt )[0]
#         efx = np.where( exp_fil[ sub_expind[esx] ] == ifilt )[0]
#         j = i + 3

#         if row == 0:
#             ax[row,i].plot( sub_lmg[csx[cfx]], sub_nobjg[csx[cfx]]/sub_nobj[csx[cfx]], 'o', ms=3, alpha=0.02, mew=0, color=c[i])
#             ax[row,i].plot( sub_lmg[esx[efx]], sub_nobjg[esx[efx]]/sub_nobj[esx[efx]], 'o', ms=3, alpha=0.02, mew=0, color=c[j])
#             ax[row,i].set_xlabel("Limiting Magnitude")
#         if row == 1:
#             ax[row,i].plot( sub_see[csx[cfx]], sub_nobjg[csx[cfx]]/sub_nobj[csx[cfx]], 'o', ms=3, alpha=0.02, mew=0, color=c[i])
#             ax[row,i].plot( sub_see[esx[efx]], sub_nobjg[esx[efx]]/sub_nobj[esx[efx]], 'o', ms=3, alpha=0.02, mew=0, color=c[j])
#             ax[row,i].set_xlabel("Seeing")
#         if row == 2:
#             ax[row,i].plot( sub_sks[csx[cfx]], sub_nobjg[csx[cfx]]/sub_nobj[csx[cfx]], 'o', ms=3, alpha=0.02, mew=0, color=c[i])
#             ax[row,i].plot( sub_sks[esx[efx]], sub_nobjg[esx[efx]]/sub_nobj[esx[efx]], 'o', ms=3, alpha=0.02, mew=0, color=c[j])
#             ax[row,i].set_xlabel("Sky Background")
#         if row == 3:
#             ax[row,i].plot( exp_mjd[sub_expind[csx[cfx]]], sub_nobjg[csx[cfx]]/sub_nobj[csx[cfx]], 'o', ms=3, alpha=0.02, mew=0, \
#                            color=c[i])
#             ax[row,i].plot( exp_mjd[sub_expind[esx[efx]]], sub_nobjg[esx[efx]]/sub_nobj[esx[efx]], 'o', ms=3, alpha=0.02, mew=0, \
#                            color=c[j])
#             ax[row,i].set_xlabel("MJD")
#         if row == 4:
#             ax[row,i].plot( sub_ccd[csx[cfx]], sub_nobjg[csx[cfx]]/sub_nobj[csx[cfx]], 'o', ms=3, alpha=0.02, mew=0, color=c[i])
#             ax[row,i].plot( sub_ccd[esx[efx]], sub_nobjg[esx[efx]]/sub_nobj[esx[efx]], 'o', ms=3, alpha=0.02, mew=0, color=c[j])
#             ax[row,i].set_xlabel("CCD")
#         if row == 5:
#             ax[row,i].plot( sub_nobj[csx[cfx]], sub_nobjg[csx[cfx]]/sub_nobj[csx[cfx]], 'o', ms=3, alpha=0.02, mew=0, color=c[i])
#             ax[row,i].plot( sub_nobj[esx[efx]], sub_nobjg[esx[efx]]/sub_nobj[esx[efx]], 'o', ms=3, alpha=0.02, mew=0, color=c[j])
#             ax[row,i].set_xlabel("Number of Objects (all)")
#             ax[row,i].set_xlim([0,100])

#         ax[row,i].yaxis.set_tick_params(which='both', labelbottom=True)
#         if i == 0:
#             ax[row,i].set_ylabel("Fraction of Good Objects")
#         # ax[row,i].set_ylim([0,20])

#         del cfx,efx,j
    
# fig.tight_layout()
# plt.show()

<br>
<br>

## plot object qualities

### index objects by field

In [ ]:
cox = np.where( (obj_ra > 140.0) & (obj_ra < 160.0) & (obj_dec > -5.0)  & (obj_dec < 10.0) )[0]
eox = np.where( (obj_ra > 0.0)   & (obj_ra < 20.0)  & (obj_dec > -50.0) & (obj_dec < -40.0) )[0]
print('Number of COSMOS detections: ',len(cox))
print('Number of ELAIS detections: ',len(eox))

rbg_cox = np.where( (obj_rb > 0.6)  & (obj_ra > 140.0) & (obj_ra < 160.0) & \
                   (obj_dec > -5.0)  & (obj_dec < 10.0) )[0]
rbg_eox = np.where( (obj_rb > 0.6) & (obj_ra > 0.0)   & (obj_ra < 20.0)  & \
                   (obj_dec > -50.0) & (obj_dec < -40.0) )[0]
print('Number of COSMOS detections with r/b>0.6: ',len(rbg_cox))
print('Number of ELAIS detections with r/b>0.6: ',len(rbg_eox))

#### object real/bogus distributions per filter

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(16,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( exp_fil[ sub_expind[ obj_subind[cox] ] ] == ifilt )[0]
    ax[i].hist( obj_rb[cox[fcx]], bins=20, log=True, histtype='step', lw=2, \
               color=c[i], label='COSMOS, '+ifilt+'-band')

    j = i + 3
    fex = np.where( exp_fil[ sub_expind[ obj_subind[eox] ] ] == ifilt )[0]
    ax[i].hist( obj_rb[eox[fex]], bins=20, log=True, histtype='step', lw=2, \
               color=c[j], label='ELAIS, '+ifilt+'-band')
    
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].legend(loc='upper right')
    ax[i].set_ylim([500,1000000])

    if i == 0:
        ax[i].set_ylabel("log(Number of Objects)")
    ax[i].set_xlabel("Real/Bogus Score")
    ax[i].axvline(0.6, color='grey')

    del fcx
    del fex
    
fig.tight_layout()
plt.savefig("source_detection_summary_figures/srcdet_RBhist.png")

#### object magnitude distributions per filter

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(16,5), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)

    fcx = np.where( exp_fil[ sub_expind[ obj_subind[cox] ] ] == ifilt )[0]
    ax[i].hist( obj_mag[cox[fcx]], bins=20, log=True, histtype='step', lw=1, \
               color=c[i], label='COSMOS (all)')
    del fcx
    
    j = i + 3
    fex = np.where( exp_fil[ sub_expind[ obj_subind[eox] ] ] == ifilt )[0]
    ax[i].hist( obj_mag[eox[fex]], bins=20, log=True, histtype='step', lw=1, \
               color=c[j], label='ELAIS (all)')
    del fex

    fcx = np.where( exp_fil[ sub_expind[ obj_subind[rbg_cox] ] ] == ifilt )[0]
    ax[i].hist( obj_mag[rbg_cox[fcx]], bins=20, log=True, histtype='step', lw=3, \
               color=c[i], label='COSMOS (R/B>0.6)')
    del fcx
    
    j = i + 3
    fex = np.where( exp_fil[ sub_expind[ obj_subind[rbg_eox] ] ] == ifilt )[0]
    ax[i].hist( obj_mag[rbg_eox[fex]], bins=20, log=True, histtype='step', lw=3, \
               color=c[j], label='ELAIS (R/B>0.6)')
    del fex

    if i == 0:
        ax[i].set_ylabel("Number of Objects")
    ax[i].set_xlabel("Apparent Magnitude")
    ax[i].axvline(23.5, color='grey')
    ax[i].legend(loc='upper center', bbox_to_anchor=(0.45, 1.25), ncol=2, fontsize=13)

fig.tight_layout()
plt.savefig("source_detection_summary_figures/srcdet_maghist.png")

### real/bogus vs. magnitude

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(12,3), sharey=True )
for i,ifilt in enumerate(['g','r','i']):

    fcx = np.where( exp_fil[ sub_expind[ obj_subind[cox] ] ] == ifilt )[0]
    ax[i].plot( obj_mag[cox[fcx]], obj_rb[cox[fcx]], 'o', ms=2, alpha=0.2, mew=0, color=c[i])
    del fcx

    j = i + 3
    fex = np.where( exp_fil[ sub_expind[ obj_subind[eox] ] ] == ifilt )[0]
    ax[i].plot( obj_mag[eox[fex]], obj_rb[eox[fex]], 'o', ms=2, alpha=0.2, mew=0, color=c[j])
    del fex

    if i == 0:
        ax[i].set_ylabel("R/B")
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].set_xlabel("mag")

fig.tight_layout()
plt.show()

<br>
<br>

## plot candidate qualities

### index candidates by field

In [ ]:
ccx = np.where( (cand_ra > 140.0) & (cand_ra < 160.0) & (cand_dec > -5.0)  & (cand_dec < 10.0) )[0]
ecx = np.where( (cand_ra > 0.0)   & (cand_ra < 20.0)  & (cand_dec > -50.0) & (cand_dec < -40.0) )[0]
print('Number of COSMOS candidates: ',len(ccx))
print('Number of ELAIS candidates: ',len(ecx))

# likely real based on mean real/bogus score
cut_RB = 0.6
str_cut_RB = str(cut_RB)
rbg_ccx = np.where( (cand_meanrb > cut_RB) & \
                   (cand_ra > 140.0) & (cand_ra < 160.0) & \
                   (cand_dec > -5.0)  & (cand_dec < 10.0) )[0]
rbg_ecx = np.where( (cand_meanrb > cut_RB) & \
                   (cand_ra > 0.0)   & (cand_ra < 20.0)  & \
                   (cand_dec > -50.0) & (cand_dec < -40.0) )[0]
print('Number of COSMOS candidates with mean r/b>0.6: ',len(rbg_ccx))
print('Number of ELAIS candidates with mean r/b>0.6: ',len(rbg_ecx))

# likely real based on number of detections
cut_Nobj = 10
str_cut_Nobj = str(cut_Nobj)
rdt_ccx = np.where( (cand_nobj > cut_Nobj) & \
                   (cand_ra > 140.0) & (cand_ra < 160.0) & \
                   (cand_dec > -5.0)  & (cand_dec < 10.0) )[0]
rdt_ecx = np.where( (cand_nobj > cut_Nobj) & \
                   (cand_ra > 0.0)   & (cand_ra < 20.0)  & \
                   (cand_dec > -50.0) & (cand_dec < -40.0) )[0]
print('Number of COSMOS candidates with nobj>30: ',len(rdt_ccx))
print('Number of ELAIS candidates with nobj>30: ',len(rdt_ecx))

# likely real based on both mean real/bogus and number of detections = LIKELY GOOD
good_ccx = np.where( (cand_nobj > cut_Nobj) & (cand_meanrb > cut_RB) & \
                    (cand_ra > 140.0) & (cand_ra < 160.0) & \
                    (cand_dec > -5.0)  & (cand_dec < 10.0) )[0]
good_ecx = np.where( (cand_nobj > cut_Nobj) & (cand_meanrb > cut_RB) & \
                    (cand_ra > 0.0)   & (cand_ra < 20.0) & \
                    (cand_dec > -50.0) & (cand_dec < -40.0) )[0]
print('Number of COSMOS candidates that are "good": ',len(good_ccx))
print('Number of ELAIS candidates that are "good": ',len(good_ecx))

#### show the spatial distribution of candidates

In [ ]:
fig, ax = plt.subplots( 2, 4, figsize=(16,6), sharey=False )

ax[0,0].plot( cand_ra[ccx], cand_dec[ccx], 'o', ms=2, alpha=0.01, mew=0, color=cc)
ax[0,1].plot( cand_ra[rbg_ccx], cand_dec[rbg_ccx], 'o', ms=2, alpha=0.2, mew=0, color=cc)
ax[0,2].plot( cand_ra[rdt_ccx], cand_dec[rdt_ccx], 'o', ms=2, alpha=0.2, mew=0, color=cc)
ax[0,3].plot( cand_ra[good_ccx], cand_dec[good_ccx], 'o', ms=4, alpha=0.5, mew=0, color=cc)

ax[1,0].plot( cand_ra[ecx], cand_dec[ecx], 'o', ms=2, alpha=0.01, mew=0, color=ce)
ax[1,1].plot( cand_ra[rbg_ecx], cand_dec[rbg_ecx], 'o', ms=2, alpha=0.2, mew=0, color=ce)
ax[1,2].plot( cand_ra[rdt_ecx], cand_dec[rdt_ecx], 'o', ms=2, alpha=0.2, mew=0, color=ce)
ax[1,3].plot( cand_ra[good_ecx], cand_dec[good_ecx], 'o', ms=4, alpha=0.5, mew=0, color=ce)

ax[0,0].set_title( 'all' )
ax[0,1].set_title( 'mean R/B > '+str_cut_RB )
ax[0,2].set_title( '#obj > '+str_cut_Nobj )
ax[0,3].set_title( 'good candidates' )
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots( 1, 2, figsize=(16,5), sharey=True )

ax[0].axvline( cut_RB, color='grey' )
ax[0].hist( cand_meanrb[ccx],     bins=20, log=True, histtype='step', lw=1, color=cc, \
           label='COSMOS (all)')
ax[0].hist( cand_meanrb[ecx],     bins=20, log=True, histtype='step', lw=1, color=ce, \
           label='ELAIS (all)')
ax[0].hist( cand_meanrb[rdt_ccx], bins=20, log=True, histtype='step', lw=3, color=cc, \
           label='COSMOS (Nobj>'+str_cut_Nobj+')')
ax[0].hist( cand_meanrb[rdt_ecx], bins=20, log=True, histtype='step', lw=3, color=ce, \
           label='ELAIS (Nobj>'+str_cut_Nobj+')')
ax[0].set_xlabel("Mean R/B Score")
ax[0].set_ylabel("Number of Candidates")

ax[1].axvline( np.log10(cut_Nobj), color='grey' )
ax[1].hist( np.log10(cand_nobj[ccx]),     bins=40, range=(0,3.0), log=True, \
           histtype='step', lw=1, color=cc, label='COSMOS (all)')
ax[1].hist( np.log10(cand_nobj[ecx]),     bins=40, range=(0,3.0), log=True, \
           histtype='step', lw=1, color=ce, label='ELAIS (all)')
ax[1].hist( np.log10(cand_nobj[rbg_ccx]), bins=40, range=(0,3.0), log=True, \
           histtype='step', lw=3, color=cc, label='COSMOS (mean R/B>'+str_cut_RB+')')
ax[1].hist( np.log10(cand_nobj[rbg_ecx]), bins=40, range=(0,3.0), log=True, \
           histtype='step', lw=3, color=ce, label='ELAIS (mean R/B>'+str_cut_RB+')')
ax[1].set_xlabel("Number of Objects")
ax[1].set_xlim([-0.1,2.9])
# ax[1].set_ylabel("Number of Candidates")

ax[1].set_ylim([1,1000000])

for i in (0,1):
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
ax[0].legend(loc='upper center', bbox_to_anchor=(0.34, 1.25), ncol=2, fontsize=13)
ax[1].legend(loc='upper center', bbox_to_anchor=(0.37, 1.25), ncol=2, fontsize=13)

fig.tight_layout()
plt.savefig("source_detection_summary_figures/srcdet_candhists.png")

In [ ]:
fig, ax = plt.subplots( 1, 2, figsize=(12,4), sharey=True )
ax[0].yaxis.set_tick_params(which='both', labelbottom=True)
ax[0].plot( cand_meanrb[good_ccx], cand_nobj[good_ccx], 'o', ms=7, \
           alpha=0.2, mew=0, color=cc, label='COSMOS')
# ax[0].legend()
ax[0].set_xlabel("mean real/bogus")
ax[0].set_ylabel("number of detections")
ax[1].yaxis.set_tick_params(which='both', labelbottom=True)
ax[1].plot( cand_meanrb[good_ecx], cand_nobj[good_ecx], 'o', ms=7, \
           alpha=0.2, mew=0, color=ce, label='ELAIS')
# ax[1].legend()
ax[1].set_xlabel("mean real/bogus")
fig.tight_layout()
# plt.savefig("./source_detection_summary_figures/cand_mrb_nobj.png")

## Read in the data tables and store as numpy arrays
Note that in some cases specifying the type throws an error, and so type is omitted for some numpy arrays.

In [ ]:
# t1 = time.time()

# query = "SELECT id, filename, mjd, filter FROM exposures"
# cursor.execute( query )
# results = np.array( cursor.fetchall() ).transpose()

# t2 = time.time()
# print(t2-t1)
# del t1,t2

# exp_id  = np.asarray( results[0], dtype='int' )
# exp_fnm = np.asarray( results[1], dtype='str' )
# exp_mjd = np.asarray( results[2], dtype='float' )
# exp_fil = np.asarray( results[3], dtype='str' )
# del query, results

# print(len(exp_id))

In [ ]:
# t1 = time.time()

# query = "SELECT id, lmt_mg, seeing, skysig, magzp, exposure_id FROM subtractions"
# cursor.execute( query )
# results = np.array( cursor.fetchall() ).transpose()

# t2 = time.time()
# print(t2-t1)
# del t1,t2

# sub_id     = np.asarray( results[0], dtype='int' )
# sub_lmtmg  = np.asarray( results[1], dtype='float' )
# sub_seeing = np.asarray( results[2], dtype='float' )
# sub_skysig = np.asarray( results[3], dtype='float' )
# sub_magzp  = np.asarray( results[4], dtype='float' )
# sub_expid  = np.asarray( results[5], dtype='int' )
# del query, results

# print(len(sub_id))

In [ ]:
# t1 = time.time()

# query = ("SELECT o.id, o.ra, o.dec, o.candidate_id, o.subtraction_id, rbs.rb, o.mag, o.magerr, o.ignore FROM objects o "
#          "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=1 ")
# cursor.execute( query )
# results = np.array( cursor.fetchall() ).transpose()

# t2 = time.time()
# print(t2-t1)
# del t1,t2

# obj_id     = np.asarray( results[0], dtype='int' )
# obj_ra     = np.asarray( results[1], dtype='float' )
# obj_dec    = np.asarray( results[2], dtype='float' )
# obj_candid = np.asarray( results[3] )
# obj_subid  = np.asarray( results[4], dtype='int' )
# obj_rb     = np.asarray( results[5], dtype='float' )
# obj_mag    = np.asarray( results[6], dtype='float' )
# obj_magerr = np.asarray( results[7], dtype='float' )
# obj_ignore = np.asarray( results[8] )
# del query, results

# print(len(obj_id))

In [ ]:
### What is this obj_ignore array?
# ix = np.where( obj_ignore == True )[0]
# print(len(ix))

In [ ]:
# t1 = time.time()

# query = "SELECT id, ra, dec, nmatches FROM candidates"
# cursor.execute( query )
# results = np.array( cursor.fetchall() ).transpose()

# t2 = time.time()
# print(t2-t1)
# del t1,t2

# cand_id       = np.asarray( results[0] )
# cand_ra       = np.asarray( results[1], dtype='float' )
# cand_dec      = np.asarray( results[2], dtype='float' )
# cand_nmatches = np.asarray( results[3] )
# del query, results

# print(len(cand_id))

In [ ]:
# ### There are more cand_id than there are unique obj_candid values.
# print(len(np.unique(cand_id)))
# print(len(np.unique(obj_candid)))

### What is cand_nmatches??

It appears to be empty.

In [ ]:
# print(cand_nmatches)
# print(len(cand_nmatches))
# cx = np.where( cand_nmatches != None )[0]
# print(len(cx))
# print(cand_nmatches[cx])

#### No matter, we can easily get the number of unique candidate ids from the object array

In [ ]:
# #all 
# values, indices, counts = np.unique( obj_candid, return_inverse = True, return_counts = True )
# print( len(values), len(indices), len(counts) )
# print( values )
# print( indices )
# print( counts )
# print( np.min(counts), np.max(counts) )

# #good "likely-real"
# gx = np.where( obj_rb > 0.6 )[0]
# gvalues, gindices, gcounts = np.unique( obj_candid[gx], return_inverse = True, return_counts = True )
# print( len(gvalues), len(gindices), len(gcounts) )
# print( gvalues )
# print( gindices )
# print( gcounts )
# print( np.min(gcounts), np.max(gcounts) )

In [ ]:
# ### Plot a histogram of the number of candidates with N objects
# plt.figure(figsize=(12, 4))
# plt.hist( counts, bins=100, log=True )
# plt.xlabel('number of objects')
# plt.ylabel('log(number of candidates)')
# plt.show()

In [ ]:
# ### Plot a histogram of the number of candidates with N good objects
# plt.figure(figsize=(12, 4))
# plt.hist( gcounts, bins=100, log=True, color='darkorange' )
# plt.xlabel('number of good objects')
# plt.ylabel('log(number of candidates)')
# plt.show()

#### And then make new array to hold the number of objects for a candidate

In [ ]:
# ### The number of objects associated with the same candidate, for each object
# obj_nobj = counts[indices]
# print(obj_nobj)

#### And also a new cand_nmatches, and one for just "good" objects too

In [ ]:
### Get rid of the useless cand_nmatches and repopulate it

# t1 = time.time()

# del cand_nmatches
# cand_nmatches = np.zeros( len(cand_id), dtype='int' )
# cand_ngmatches = np.zeros( len(cand_id), dtype='int' )
# sx = np.argsort( cand_id )

# ### Using sorted lists is the only way to do this is a reasonable amount of time
# ### This is a loop for every unique candidate id
# cntr = 0
# for c,val in enumerate( values ):
#     if (c==100) | (c==1000) | (c==10000):
#         print( 'c = ', c, np.round(time.time()-t1,3), 's elapsed' )

#     match = False
#     while match == False:
#         if val == cand_id[sx[cntr]]:
#             cand_nmatches[sx[cntr]] = counts[c]
#             cntr += 1
#             match = True
#         else:
#             cntr += 1

            
# cntr = 0
# for c,gval in enumerate( gvalues ):
#     if (c==100) | (c==1000) | (c==10000):
#         print( 'c = ', c, np.round(time.time()-t1,3), 's elapsed' )

#     match = False
#     while match == False:
#         if gval == cand_id[sx[cntr]]:
#             cand_ngmatches[sx[cntr]] = gcounts[c]
#             cntr += 1
#             match = True
#         else:
#             cntr += 1

# t2 = time.time()
# print(t2-t1)
# del t1,t2

In [ ]:
### Just double check that values of cand_nmatches are as expected (similar to counts array)
# print( cand_nmatches )
# print( np.min(cand_nmatches), np.max(cand_nmatches) )
# for n in [10,20,30,100]:
#     print(n, len(np.where( cand_nmatches >= n )[0]) )

In [ ]:
# print( cand_ngmatches )
# print( np.min(cand_ngmatches), np.max(cand_ngmatches) )
# for n in [10,20,30,100]:
#     print(n, len(np.where( cand_ngmatches >= n )[0]) )

In [ ]:
### Plot a histogram of the number of candidates with N objects
# plt.figure(figsize=(12, 4))
# plt.hist( cand_nmatches, bins=100, log=True )
# plt.xlabel('number of objects')
# plt.ylabel('log(number of candidates)')
# plt.show()

In [ ]:
del values, indices, counts

### Flag objects that are duplicates
They are within 0.1" of another object in the same subtraction.<br>
This takes about 600 seconds.

In [ ]:
# t1 = time.time()

# obj_dup = np.zeros( len(obj_id), dtype='int' )

# limit = (0.1 / 3600.0)**2
# for s,subid in enumerate(sub_id):
#     if (s==100) | (s==1000) | (s==10000) | (s==30000) | (s==60000):
#         ix = np.where( obj_dup == 1 )[0]
#         print( 's = ', s, np.round(time.time()-t1,3), 's elapsed, ', len(ix),' duplicates identified' )
#         del ix
#     # all the objects in this subtraction
#     ox = np.where( (obj_subid == subid) & (obj_dup == 0) )[0]
#     for o in ox:
#         # all the objects in this subtraction with similar RA and Dec to the o'th in ox
#         tx = np.where( (obj_ra[ox]-obj_ra[o])**2 + (obj_dec[ox]-obj_dec[o])**2 < limit )[0]
#         if len(tx) > 0:
#             # identify all the matching objects as duplicates
#             obj_dup[ox[tx]] = 1
#             # except the first one, of course, consider it the 'real' one
#             obj_dup[o] = 0
#         del tx
#     del ox

# tx = np.where( obj_dup == 1 )[0]
# print('total number of duplicates identified: ',len(tx))
# del tx

# t2 = time.time()
# print(t2-t1)
# del t1,t2

### Flag objects that are from "bad" (short-exposure) images
(i.e., files that do not have a classification of COSMOS, COSMOS-AGN, or ELAIS)

At the same time, get the filter for each object.

In [ ]:
# temp = []
# for efnm in exp_fnm:
#     temp.append( efnm[0:21] )
# exp_fnmbase = np.asarray( temp, dtype='str' )
# print( exp_fnm[0], exp_fnmbase[0] )

In [ ]:
# t1 = time.time()

# obj_bimg = np.zeros( len(obj_id), dtype='int' )
# obj_filt = np.zeros( len(obj_id), dtype='str' )
# obj_mjd  = np.zeros( len(obj_id), dtype='float' )

# img_fnmbase = np.loadtxt( 'archive_image_list.txt', dtype='str', usecols=(0) )
# img_class   = np.loadtxt( 'archive_image_list.txt', dtype='str', usecols=(9) )

# for f,fnmbase in enumerate(img_fnmbase):
#     if (f==10) | (f==100) | (f==1000):
#         ix = np.where( obj_bimg == 1 )[0]
#         print( 'f = ', f, np.round(time.time()-t1,3), 's elapsed, ', len(ix),' objects in bad images identified' )
#         del ix
#     if (img_class[f] != 'COSMOS') & (img_class[f] != 'COSMOS-AGN') & (img_class[f] != 'ELAIS'):
#         # all the exposures associated with this filename
#         ex = np.where( exp_fnmbase == fnmbase )[0]
#         if len(ex) > 0:
#             for e in ex:
#                 # all the subtractions associated with this exposure
#                 sx = np.where( sub_expid == exp_id[e] )[0]
#                 if len(sx) > 0:
#                     for s in sx:
#                         # all the objects associated with this subtraction
#                         ox = np.where( obj_subid == sub_id[s] )[0]
#                         if len(ox) > 0:
#                             obj_bimg[ox] = 1
#                         del ox
#                 del sx
#         del ex
#     else:
#         # all the exposures associated with this filename
#         ex = np.where( exp_fnmbase == fnmbase )[0]
#         if len(ex) > 0:
#             for e in ex:
#                 sx = np.where( sub_expid == exp_id[e] )[0]
#                 if len(sx) > 0:
#                     for s in sx:
#                         # all the objects associated with this subtraction
#                         ox = np.where( obj_subid == sub_id[s] )[0]
#                         if len(ox) > 0:
#                             obj_filt[ox] = exp_fil[e]
#                             obj_mjd[ox] = exp_mjd[e]
#                         del ox
#                 del sx
#         del ex

# tx = np.where( obj_bimg == 1 )[0]
# print('number of objects in bad images = ', len(tx) )
# del tx

# t2 = time.time()
# print(t2-t1)
# del t1,t2

In [ ]:
### Just make sure the values of obj_filt are as expected
# tx = np.where( obj_bimg == 0 )[0]
# print( obj_filt[tx] )
# print( np.unique( obj_filt[tx] ) )
# del tx

### Generate some features of candidates (with >10 good matches) light curves.

Candidate light curve time span (tspan), amplitude (amp) and mean R/B score. Only for objects (detections) with R/B>0.6.

The following takes about 4 minutes.

In [ ]:
t1 = time.time()

### Candidate LC timespan and amplitude
###  dimension 0,1,2,3 are for all filters, g, r, and i
cand_LCtspan = np.zeros( (len(cand_nmatches),4), dtype='float' )
cand_LCamp   = np.zeros( (len(cand_nmatches),4), dtype='float' )
cand_LCngdet = np.zeros( (len(cand_nmatches),4), dtype='int' )
cand_meanrb  = np.zeros( (len(cand_nmatches),4), dtype='float' )

cx = np.where( cand_ngmatches >=10 )[0]
print('len(cx) = ', len(cx) )

for i,c in enumerate(cx):
    if (i==100) | (i==1000) | (i==10000):
        print( 'i = ', i, 'c = ', c, np.round(time.time()-t1,3), 's elapsed' )

    ### All non-bad_img, non-duplicate, likely-real objects associated with this candidate
    ox = np.where( (obj_candid == cand_id[c]) & (obj_bimg == 0) & (obj_dup == 0) & (obj_rb > 0.6) )[0]
    if len(ox) >= 10:
        cand_LCtspan[c,0] = np.max(obj_mjd[ox]) - np.min(obj_mjd[ox])
        cand_LCamp[c,0]   = np.max(obj_mag[ox]) - np.min(obj_mag[ox])
        cand_LCngdet[c,0] = len(ox)
        cand_meanrb[c,0]  = np.mean(obj_rb[ox])
        for f,fil in enumerate(['g','r','i']):
            fx = np.where( obj_filt[ox] == fil )[0]
            if len(fx) > 1:
                cand_LCtspan[c,f+1] = np.max(obj_mjd[ox[fx]]) - np.min(obj_mjd[ox[fx]])
                cand_LCamp[c,f+1]   = np.max(obj_mag[ox[fx]]) - np.min(obj_mag[ox[fx]])
                cand_LCngdet[c,f+1] = len(fx)
                cand_meanrb[c,f+1]  = np.mean(obj_rb[ox[fx]])
            del fx
    del ox

print( 'finished. ', np.round(time.time()-t1,3), 's elapsed' )
del t1,cx

<br><br>

## Plots for ALL the objects 
Except those in "bad" images, or objects which are duplicates

In [ ]:
c = ['darkgreen', 'red', 'brown', 'limegreen', 'darkorange', 'peru']

In [ ]:
cx = np.where( (obj_bimg == 0) & (obj_dup == 0) & (obj_ra > 145.0) & (obj_ra < 155.0) & (obj_dec > -3.0) & (obj_dec < 7.0) )[0]
ex = np.where( (obj_bimg == 0) & (obj_dup == 0) & (obj_ra > 3.0) & (obj_ra < 13.0) & (obj_dec > -48.0) & (obj_dec < -38.0) )[0]
print(len(cx),len(ex))

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[cx] == ifilt )[0]
    fex = np.where( obj_filt[ex] == ifilt )[0]
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].hist( [obj_rb[cx[fcx]],obj_rb[ex[fex]]], bins=20, stacked=True, log=True, color=[c[i],c[i+3]], label=['COSMOS','ELAIS'])
    ax[i].legend()
    if i == 0:
        ax[i].set_ylabel("number of detections")
    ax[i].set_xlabel("real/bogus score")
    ax[i].set_ylim([1e3,8e5])
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/rbhist.png")

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[cx] == ifilt )[0]
    fex = np.where( obj_filt[ex] == ifilt )[0]
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].hist( [obj_mag[cx[fcx]],obj_mag[ex[fex]]], bins=20, stacked=True, log=True, color=[c[i],c[i+3]], label=['COSMOS','ELAIS'])
#     ax[i].legend()
    if i == 0:
        ax[i].set_ylabel("number of detections")
    ax[i].set_xlabel("apparent magnitude")
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/maghist.png")

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[cx] == ifilt )[0]
    fex = np.where( obj_filt[ex] == ifilt )[0]
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].plot( obj_rb[cx[fcx]], obj_mag[cx[fcx]], 'o',ms=2,alpha=0.01,mew=0, color=c[i], label='COSMOS' )
    ax[i].plot( obj_rb[ex[fex]], obj_mag[ex[fex]], 'o',ms=2,alpha=0.03,mew=0, color=c[i+3], label='ELAIS' )
    if i == 0:
        ax[i].set_ylabel("apparent magnitude")
    ax[i].set_xlabel("real/bogus score")
    ax[i].set_ylim([15.0,24.0])
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/magVrb.png")

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[cx] == ifilt )[0]
    fex = np.where( obj_filt[ex] == ifilt )[0]
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].plot( obj_rb[cx[fcx]], np.log10(obj_nobj[cx[fcx]]), 'o',ms=2,alpha=0.01,mew=0, color=c[i], label='COSMOS' )
    ax[i].plot( obj_rb[ex[fex]], np.log10(obj_nobj[ex[fex]]), 'o',ms=2,alpha=0.03,mew=0, color=c[i+3], label='ELAIS' )
    if i == 0:
        ax[i].set_ylabel("log(associated detections)")
    ax[i].set_xlabel("real/bogus score")
#     ax[i].set_ylim([15.0,24.0])
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/nobjVrb.png")

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( (obj_filt[cx] == ifilt) & (obj_rb[cx] > 0.9) )[0]
    fex = np.where( (obj_filt[ex] == ifilt) & (obj_rb[ex] > 0.9) )[0]
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].hist( [np.log10(obj_nobj[cx[fcx]]),np.log10(obj_nobj[ex[fex]])], bins=20, \
               stacked=True, color=[c[i],c[i+3]], label=['COSMOS','ELAIS'] )
    if i == 0:
        ax[i].set_ylabel("# of det. (R/B > 0.9)")
    ax[i].set_xlabel("log(associated detections)")
#     ax[i].set_ylim([15.0,24.0])
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/nobjcutoff.png")

<br><br>

## Plots for Objects that are likely real (R/B>0.6) and potentially astrophysical (>10 detections of any R/B value).

In [ ]:
good_cx = np.where( (obj_nobj > 10) & (obj_rb >= 0.6) & (obj_bimg == 0) & (obj_dup == 0) & \
                   (obj_ra > 145.0) & (obj_ra < 155.0) & (obj_dec > -3.0) & (obj_dec < 7.0) )[0]
good_ex = np.where( (obj_nobj > 10) & (obj_rb >= 0.6) & (obj_bimg == 0) & (obj_dup == 0) & \
                   (obj_ra > 3.0) & (obj_ra < 13.0) & (obj_dec > -48.0) & (obj_dec < -38.0) )[0]

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[good_cx] == ifilt )
    fex = np.where( obj_filt[good_ex] == ifilt )
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].plot( obj_rb[good_cx[fcx]], obj_mag[good_cx[fcx]], 'o',ms=2,alpha=0.01,mew=0, color=c[i], label='COSMOS' )
    ax[i].plot( obj_rb[good_ex[fex]], obj_mag[good_ex[fex]], 'o',ms=2,alpha=0.1,mew=0, color=c[i+3], label='ELAIS' )
    if i == 0:
        ax[i].set_ylabel("apparent magnitude")
    ax[i].set_xlabel("real/bogus score")
    ax[i].set_ylim([15.0,24.0])
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/magVrb_good.png")

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[good_cx] == ifilt )
    fex = np.where( obj_filt[good_ex] == ifilt )
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].hist( [obj_mag[good_cx[fcx]],obj_mag[good_ex[fex]]], bins=20, stacked=True, color=[c[i],c[i+3]], label=['COSMOS','ELAIS'])
    ax[i].legend()
    if i == 0:
        ax[i].set_ylabel("log(number of detections)")
    ax[i].set_xlabel("apparent magnitude")
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/maghist_good.png")

In [ ]:
nbins = int(np.ceil( np.max(obj_mjd[good_cx]) - np.min(obj_mjd[good_cx]) ))
print(nbins)
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[good_cx] == ifilt )
    fex = np.where( obj_filt[good_ex] == ifilt )
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].hist( [obj_mjd[good_cx[fcx]],obj_mjd[good_ex[fex]]], bins=nbins, stacked=True, color=[c[i],c[i+3]], label=['COSMOS','ELAIS'])
    ax[i].legend()
    ax[i].set_xlabel("MJD")
    if i == 0:
        ax[i].set_ylabel("log(number of detections)")
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/maghist_good.png")

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fcx = np.where( obj_filt[good_cx] == ifilt )
    fex = np.where( obj_filt[good_ex] == ifilt )
    ax[i].yaxis.set_tick_params(which='both', labelbottom=True)
    ax[i].plot( obj_mag[good_cx[fcx]],obj_magerr[good_cx[fcx]], 'o',ms=2,alpha=0.05, color=c[i])
    ax[i].plot( obj_mag[good_ex[fex]],obj_magerr[good_ex[fex]], 'o',ms=2,alpha=0.05, color=c[i+3])
#     ax[i].legend()
    if i == 0:
        ax[i].set_ylabel("error")
    ax[i].set_xlabel("apparent magnitude")
    ax[i].set_xlim([17,24])
    ax[i].set_ylim([0.0,0.3])
    del fcx,fex
fig.tight_layout()
plt.savefig("./source_detection_summary_figures/magerr_good.png")

## Plots for Candidates that are likely supernovae

They had >10 good (R/B>0.6) detections overall. Then we impose >10 good (R/B>0.6) in each filter as well, for the plots.

In [ ]:
cx = np.where( cand_ngmatches >=10 )[0]

Plot the amplitude vs. the time span. Notice how setting amplitude>0.6 mag and timespan 15-70 days cuts out a lot of points that are clustered in the timespan axis. Setting a timepan limit of 80 days also rejects candidates that had simply changed from the time of the template image but did not vary during our survey time (like AGN, which is a real astrophysical transient but not a SN).

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    ax[i].axhline(0.6, alpha=0.5,color='grey')
    ax[i].axvline(15, alpha=0.5,color='grey',ls='dashed')
    ax[i].axvline(80, alpha=0.5,color='grey',ls='dashed')
    fx = np.where( cand_LCngdet[cx,i+1] >= 10 )[0]
    randos = np.random.normal( size=len(fx) ) * 0.5
    ax[i].plot( cand_LCtspan[cx[fx],i+1]+randos, cand_LCamp[cx[fx],i+1], 'o',ms=4,mew=0,alpha=0.2, color=c[i] )
    del fx
    if i == 0:
        ax[i].set_ylabel("LC Amplitude [mag]")
    ax[i].set_xlabel("LC Timespan [days]")
fig.tight_layout()

Plot the mean R/B score for all associated detections with R/B>0.6. There aren't really any usable correlations here with mean R/B value that we could use.

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( cand_LCngdet[cx,i+1] >= 10 )[0]
    randos = np.random.normal( size=len(fx) ) * 0.5
    ax[i].plot( cand_LCtspan[cx[fx],i+1]+randos, cand_meanrb[cx[fx],i+1], 'o',ms=4,mew=0,alpha=0.2, color=c[i] )
    del fx
    ax[i].set_xlabel("Lightcurve Timespan [days]")
    if i == 0:
        ax[i].set_ylabel("Mean R/B (for R/B>0.6 det.)")
fig.tight_layout()

fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( cand_LCngdet[cx,i+1] >= 10 )[0]
    ax[i].plot( cand_LCamp[cx[fx],i+1], cand_meanrb[cx[fx],i+1], 'o',ms=4,mew=0,alpha=0.2, color=c[i] )
    del fx
    ax[i].set_xlabel("Lightcurve Amplitude [mag]")
    if i == 0:
        ax[i].set_ylabel("Mean R/B (for R/B>0.6 det.)")
fig.tight_layout()

#### There is some odd clustering of candidates in RA and Dec.

In [ ]:
cx = np.where( (cand_ngmatches >=10) & \
              (cand_ra > 145.0) & (cand_ra < 155.0) & (cand_dec > -3.0) & (cand_dec < 7.0) )[0]
ex = np.where( (cand_ngmatches >=10) & \
              (cand_ra > 3.0) & (cand_ra < 13.0) & (cand_dec > -48.0) & (cand_dec < -38.0) )[0]

print(' 0 < timespan < 15 days')
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( (cand_LCtspan[cx,i+1] > 0.0) & (cand_LCtspan[cx,i+1] < 15.0) )[0]
    ax[i].plot( cand_ra[cx[fx]], cand_dec[cx[fx]], 'o',ms=2,mew=0,alpha=0.2, color=c[i] )
    ax[i].set_xlabel("RA")
    if i == 0:
        ax[i].set_ylabel("Dec")
fig.tight_layout()

print(' 15 < timespan < 80 days ')
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( (cand_LCtspan[cx,i+1] >= 15.0) | (cand_LCtspan[cx,i+1] <= 80.0) )[0]
    ax[i].plot( cand_ra[cx[fx]], cand_dec[cx[fx]], 'o',ms=2,mew=0,alpha=0.2, color=c[i] )
    ax[i].set_xlabel("RA")
    if i == 0:
        ax[i].set_ylabel("Dec")
fig.tight_layout()

print(' 80 < timespan ')
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( cand_LCtspan[cx,i+1] > 80.0 )[0]
    ax[i].plot( cand_ra[cx[fx]], cand_dec[cx[fx]], 'o',ms=2,mew=0,alpha=0.2, color=c[i] )
    ax[i].set_xlabel("RA")
    if i == 0:
        ax[i].set_ylabel("Dec")
fig.tight_layout()

In [ ]:
print(' mean R/B < 0.9 ')
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( cand_meanrb[cx,i+1] < 0.9 )[0]
    ax[i].plot( cand_ra[cx[fx]], cand_dec[cx[fx]], 'o',ms=2,mew=0,alpha=0.2, color=c[i] )
    ax[i].set_xlabel("RA")
    if i == 0:
        ax[i].set_ylabel("Dec")
fig.tight_layout()

print(' mean R/B >= 0.9 ')
fig, ax = plt.subplots( 1, 3, figsize=(15,4), sharey=True )
for i,ifilt in enumerate(['g','r','i']):
    fx = np.where( cand_meanrb[cx,i+1] >= 0.9 )[0]
    ax[i].plot( cand_ra[cx[fx]], cand_dec[cx[fx]], 'o',ms=2,mew=0,alpha=0.2, color=c[i] )
    ax[i].set_xlabel("RA")
    if i == 0:
        ax[i].set_ylabel("Dec")
fig.tight_layout()